<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PG24_3F_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

PostgreSQL Tutorial https://www.w3resource.com/PostgreSQL/tutorial.php

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2025-02-27 06:49:27.836679+05:30
Python 3.11.11
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy


#Setup

In [2]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#
import psycopg2
import pandas as pd
#
!psql -V
!service postgresql start



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


psql (PostgreSQL) 14.15 (Ubuntu 14.15-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


In [3]:
# Installation creates an Ubuntu user 'postgres' AND a Postgres database user 'postgres' both of which are superusers
#!cat /etc/passwd
# Use these users to create a new database user called 'upraxis' and give it a password 'upass'
#!sudo -u postgres createuser --superuser upraxis
!sudo -u postgres createuser upraxis
!sudo -u postgres psql -U postgres -c "ALTER USER upraxis PASSWORD 'upass';"
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
# Use ubuntu superuser to create database dbpraxis and set its owner to upraxis
!sudo -u postgres createdb -O upraxis dbpraxis
# Create file to store the password. Else will be prompted each time
!echo "localhost:5432:dbpraxis:postgres:pass" > ~/.pgpass
!echo "localhost:5432:dbpraxis:upraxis:upass" >> ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass


ALTER ROLE
ALTER ROLE
localhost:5432:dbpraxis:postgres:pass
localhost:5432:dbpraxis:upraxis:upass


## Create, Load Superstore Table

In [4]:
# Create Fact Table using command line interface
#
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'drop schema if exists ss cascade'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'create schema if not exists ss'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c '\dn+'
!psql -h localhost -p 5432 -Uupraxis -d dbpraxis -c 'drop table if exists ss.order'
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
CREATE TABLE IF NOT EXISTS ss.order (\
    RowID smallint,\
    OrderID char(14),OrderDate date,\
    ShipDate date,ShipMode varchar(16),\
    CustomerID char(8),CustomerName varchar(30),Segment varchar(20),\
    Country varchar(30),City varchar(30),State varchar(30),PostalCode char(5),Region varchar(15) ,\
    ProductID varchar(20), Category varchar(40), SubCategory varchar(40), ProductName varchar(200), \
    Sales decimal(8,2), Quantity smallint, Discount decimal(4,2), Profit decimal(8,2) \
); \
"
#Describe the table ss_order
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c '\d+ ss.order'

NOTICE:  schema "ss" does not exist, skipping
DROP SCHEMA
CREATE SCHEMA
                          List of schemas
  Name  |  Owner   |  Access privileges   |      Description       
--------+----------+----------------------+------------------------
 public | postgres | postgres=UC/postgres+| standard public schema
        |          | =UC/postgres         | 
 ss     | upraxis  |                      | 
(2 rows)

NOTICE:  table "order" does not exist, skipping
DROP TABLE
CREATE TABLE
                                                       Table "ss.order"
    Column    |          Type          | Collation | Nullable | Default | Storage  | Compression | Stats target | Description 
--------------+------------------------+-----------+----------+---------+----------+-------------+--------------+-------------
 rowid        | smallint               |           |          |         | plain    |             |              | 
 orderid      | character(14)          |           |          |       

In [5]:
#Get Data File
!wget -q -O Orders.csv 'https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv'
!head Orders.csv

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136
2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582
3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714
4,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United Stat

In [6]:
# Date Format Issues
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c "set datestyle to PostgreSQL,European; show datestyle"

   DateStyle   
---------------
 Postgres, DMY
(1 row)



In [7]:
# Load Data
# Need to use U = postgres and provide password = 'pass'
!psql -h localhost -p 5432 -U postgres -d dbpraxis -c \
"set datestyle to PostgreSQL,European;copy ss.order FROM '/content/Orders.csv' DELIMITER ',' CSV HEADER;"

COPY 9994


In [8]:
!psql -h localhost -p 5432 -U upraxis -d dbpraxis -c 'select * from ss.order limit 10'
#the date data should have loaded correctly

 rowid |    orderid     | orderdate  |  shipdate  |    shipmode    | customerid |  customername   |  segment  |    country    |      city       |   state    | postalcode | region |    productid    |    category     | subcategory |                           productname                            | sales  | quantity | discount | profit  
-------+----------------+------------+------------+----------------+------------+-----------------+-----------+---------------+-----------------+------------+------------+--------+-----------------+-----------------+-------------+------------------------------------------------------------------+--------+----------+----------+---------
     1 | CA-2016-152156 | 2016-11-08 | 2016-11-11 | Second Class   | CG-12520   | Claire Gute     | Consumer  | United States | Henderson       | Kentucky   | 42420      | South  | FUR-BO-10001798 | Furniture       | Bookcases   | Bush Somerset Collection Bookcase                                | 261.96 |        2 |     0.

## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [9]:
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return

def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

In [10]:
runSelect('select * from ss.order limit 3')

rowid,orderid,orderdate,shipdate,shipmode,customerid,customername,segment,country,city,state,postalcode,region,productid,category,subcategory,productname,sales,quantity,discount,profit
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.00,41.91
2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0.00,219.58
3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0.00,6.87


In [11]:
runSelect('select max(orderdate), min(orderdate), max(shipdate), min(shipdate) from ss.order')

max,min,max,min
2017-12-30,2014-01-03,2018-01-05,2014-01-07


#ChronoTYM <br>
In Indic phraseology, tantra is knowledge, a yantra is a device to explore this knowledge and a mantra is the code that unlocks both the tantra and its yantra. Chrono/TYM a sci-fi trilogy that recreates this framework where the first novel Chronotantra defines the contours of a tech driven, futuristic society, the second, Chronoyantra, describes a device used to explore the genesis of this society, while the third, Chronomantra finally reveals the code, the key, that unlocks this puzzle.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEjdu_ajYwBRzk5iFYlcs59ZoRMMg6rhNm4JERKspvFJoveJsnfwRlzMICiC37UOja_Bd4sbfLA1pyyBbe0mj3tUfDBXoqCzNUDw3Wux7acoJBaLkWjAvJXfa7drRY2GQRAlFxEY9O7a7raY4RKy7cA3Uxgf5QsaZQ99C9GfXxFSex_rke8ROhAn41t8zOqK/s1640/tymBanner01_3.png)

In [12]:
from datetime import datetime
import pytz
print('tested',datetime.now(pytz.timezone('Asia/Calcutta')))

tested 2025-02-27 06:50:08.935960+05:30
